# ME group knockoff with CCD, block descent, and PCA

In [1]:
# load packages needed for this tutorial
using Revise
using Knockoffs
using Random
using GLMNet
using Distributions
using LinearAlgebra
using ToeplitzMatrices
using StatsBase
using CSV, DataFrames
using Plots
gr(fmt=:png);

function get_sigma(option::Int, p::Int)
    # note: groups are defined empirically within each simuation
    datadir = "/Users/biona001/Benjamin_Folder/research/4th_project_PRS/group_knockoff_test_data"
    if option == 1
        ρ = 0.7
        Σ = SymmetricToeplitz(ρ.^(0:(p-1))) |> Matrix
    elseif option == 2
        ρ = 0.7
        γ = 0.1
        groups = repeat(1:Int(p/5), inner=5)
        Σ = simulate_block_covariance(groups, ρ, γ)
    elseif option == 3
        covfile = CSV.read(joinpath(datadir, "CorG_2_127374341_128034347.txt"), DataFrame) # 3782 SNPs
        Σ = covfile |> Matrix{Float64}
        Σ = 0.99Σ + 0.01I #ensure PSD
    elseif option == 4
        df = CSV.read(joinpath(datadir, "21_37870779_38711704.csv"), DataFrame)
        Σ = df[:, 7:end] |> Matrix |> Symmetric |> Matrix
    elseif option == 5
        df = CSV.read(joinpath(datadir, "22_17674295_18295575.csv"), DataFrame)
        Σ = df[:, 7:end] |> Matrix |> Symmetric |> Matrix
    else
        error("Option should be 1-5 but was $option")
    end
    return Σ[1:p, 1:p]
end

Random.seed!(2023)
sigma_option = 5
p = 10
Σ = get_sigma(sigma_option, p)

┌ Info: Precompiling Knockoffs [878bf26d-0c49-448a-9df5-b057c815d613]
└ @ Base loading.jl:1423


10×10 Matrix{Float64}:
  1.0        -0.532991     0.103611    …   0.0766343   -0.0587515
 -0.532991    1.0         -0.205942       -0.00845909   0.0663097
  0.103611   -0.205942     1.0             0.00952024  -0.00706601
  0.387276   -0.433576     0.0048323       0.111821     0.0605837
 -0.0343489  -0.062053     0.0057773       0.0166179   -0.00705548
 -0.049941   -0.00801635   0.0140355   …  -0.0208866   -0.00453512
 -0.0455645  -0.0885772    0.0097817      -0.0291266   -0.0112117
 -0.153657   -0.217934     0.0396276      -0.0675718   -0.0141031
  0.0766343  -0.00845909   0.00952024      1.0          0.319874
 -0.0587515   0.0663097   -0.00706601      0.319874     1.0

$\Sigma$ is 10 by 10 with obvious blocks. Lets define 2 groups that capture this structure.

In [2]:
groups = repeat(1:2, inner=5)
groups

10-element Vector{Int64}:
 1
 1
 1
 1
 1
 2
 2
 2
 2
 2

Lets solve for ME knockoff using various methods

In [3]:
@time equi, _, _ = solve_s_group(Symmetric(Σ), groups, :equi)
@time me_block, _, _ = solve_s_group(Symmetric(Σ), groups, :maxent_block)
@time me_ccd, _, _ = solve_s_group(Symmetric(Σ), groups, :maxent)
@time me_pca, _, _ = solve_s_group(Symmetric(Σ), groups, :maxent_pca)
@time me_pca_zihuai, _, _ = solve_s_group(Symmetric(Σ), groups, :maxent_pca_zihuai);

 19.789519 seconds (67.23 M allocations: 3.528 GiB, 3.60% gc time, 99.94% compilation time)
 39.910552 seconds (143.36 M allocations: 8.421 GiB, 4.01% gc time, 99.08% compilation time)
  0.002222 seconds (715 allocations: 106.203 KiB)
  0.000724 seconds (577 allocations: 91.500 KiB)
[1] -7.507976
[1] -2.847747
[1] -2.835877
  1.252711 seconds (336.39 k allocations: 18.364 MiB, 10.02% compilation time)


In [4]:
equi

10×10 Matrix{Float64}:
  1.0        -0.532991   0.103611   …   0.0         0.0         0.0
 -0.532991    1.0       -0.205942       0.0         0.0         0.0
  0.103611   -0.205942   1.0            0.0         0.0         0.0
  0.387276   -0.433576   0.0048323      0.0         0.0         0.0
 -0.0343489  -0.062053   0.0057773      0.0         0.0         0.0
  0.0         0.0        0.0        …  -0.0298484  -0.0208866  -0.00453512
  0.0         0.0        0.0            0.282069   -0.0291266  -0.0112117
  0.0         0.0        0.0            1.0        -0.0675718  -0.0141031
  0.0         0.0        0.0           -0.0675718   1.0         0.319874
  0.0         0.0        0.0           -0.0141031   0.319874    1.0

In [5]:
me_block

10×10 Matrix{Float64}:
  0.935113   -0.571568    0.111285    …   0.0         0.0         0.0
 -0.571568    0.910499   -0.189641        0.0         0.0         0.0
  0.111285   -0.189641    0.996487        0.0         0.0         0.0
  0.401741   -0.375647   -0.0036184       0.0         0.0         0.0
 -0.0339824  -0.0514592   0.00374905      0.0         0.0         0.0
  0.0         0.0         0.0         …  -0.0189634   0.0284572   0.0302674
  0.0         0.0         0.0             0.183308   -0.0241293  -0.0155555
  0.0         0.0         0.0             0.728037   -0.0545385  -0.0314644
  0.0         0.0         0.0            -0.0545385   0.962494    0.303324
  0.0         0.0         0.0            -0.0314644   0.303324    0.968148

In [6]:
me_ccd

10×10 Matrix{Float64}:
  0.883753   -0.55688     0.103024    …   0.0         0.0         0.0
 -0.55688     0.903895   -0.182499        0.0         0.0         0.0
  0.103024   -0.182499    0.98141         0.0         0.0         0.0
  0.396111   -0.371722   -0.00790544      0.0         0.0         0.0
 -0.0286475  -0.0522798   0.00431598      0.0         0.0         0.0
  0.0         0.0         0.0         …  -0.0205777   0.0263136   0.0279339
  0.0         0.0         0.0             0.192815   -0.0219612  -0.012835
  0.0         0.0         0.0             0.852689   -0.0602799  -0.0313528
  0.0         0.0         0.0            -0.0602799   0.884896    0.255194
  0.0         0.0         0.0            -0.0313528   0.255194    0.878418

In [7]:
me_pca

10×10 Matrix{Float64}:
  0.883753   -0.523148    0.096995   …   0.0         0.0         0.0
 -0.523148    0.903895   -0.204573       0.0         0.0         0.0
  0.096995   -0.204573    0.98141        0.0         0.0         0.0
  0.400002   -0.420882   -0.0249711      0.0         0.0         0.0
 -0.0193344  -0.0792444  -0.0022337      0.0         0.0         0.0
  0.0         0.0         0.0        …  -0.0153156  -0.0101223   0.00470895
  0.0         0.0         0.0            0.209815   -0.0526138  -0.0251776
  0.0         0.0         0.0            0.852689   -0.0663163  -0.0445177
  0.0         0.0         0.0           -0.0663163   0.884896    0.235165
  0.0         0.0         0.0           -0.0445177   0.235165    0.878418

In [8]:
me_pca_zihuai

10×10 Matrix{Float64}:
  0.866561  -0.517335    0.106005    …   0.0         0.0         0.0
 -0.517335   0.895366   -0.201432        0.0         0.0         0.0
  0.106005  -0.201432    1.00616         0.0         0.0         0.0
  0.401182  -0.421666   -0.00414512      0.0         0.0         0.0
 -0.025057  -0.0628329   0.00683071      0.0         0.0         0.0
  0.0        0.0         0.0         …  -0.0169601  -0.0127384   0.00512339
  0.0        0.0         0.0             0.207883   -0.0384473  -0.0306304
  0.0        0.0         0.0             0.726899   -0.0743149  -0.027368
  0.0        0.0         0.0            -0.0743149   0.936898    0.2545
  0.0        0.0         0.0            -0.027368    0.2545      0.941632

Check objective:

In [9]:
m = 1
@show group_block_objective(Σ, equi, m, :maxent)
@show group_block_objective(Σ, me_block, m, :maxent_block)
@show group_block_objective(Σ, me_ccd, m, :maxent)
@show group_block_objective(Σ, me_pca, m, :maxent_pca)
@show group_block_objective(Σ, me_pca_zihuai, m, :maxent_pca_zihuai);

group_block_objective(Σ, equi, m, :maxent) = -3.3277831590859184
group_block_objective(Σ, me_block, m, :maxent_block) = -2.7965520250053992
group_block_objective(Σ, me_ccd, m, :maxent) = -2.8597010070878666
group_block_objective(Σ, me_pca, m, :maxent_pca) = -2.9115583488836547
group_block_objective(Σ, me_pca_zihuai, m, :maxent_pca_zihuai) = -2.834642876583143


## Lasso

In [ ]:
using Distributions
μ 100
μ = zeros(p)
X = rand(MvNormal(μ, Σ), n)' |> Matrix
beta = zeros(p)

## Marginal